In [5]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [6]:
URL = "https://internshala.com/internships/computer-science-internship/"
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta content="IE=9" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0 user-scalable=0" name="viewport"/>
  <meta content="702141670710132" property="fb:app_id"/>
  <meta content="website" property="og:type"/>
  <meta content="1200" property="og:image:width"/>
  <meta content="630" property="og:image:height"/>
  <link as="image" href="https://internshala.com//static/images/home/sprites/assets.png" rel="preload"/>
  <link as="image" href="https://internshala.com//static/images/internship/banners/jobs_banner_r767.png" media="(max-width: 767px)" rel="preload"/>
  <link as="image" href="https://internshala.com//static/images/internship/banners/jobs_banner_r1920.png" media="(min-width: 767.1px)" rel="preload"/>
  <meta content="@Internshala" name="twitter:site"/>
  <meta con

In [7]:
def extract_job_title_from_result(job_div, job_post):    
    flag = 0
    if(job_div.find(name="div", attrs={"class":"company"})):
        if(job_div.find(name="div", attrs={"class":"company"}).find(name="h3", attrs={"class":"profile"})):
            for a in job_div.find(name="div", attrs={"class":"company"}).find(name="h3", attrs={"class":"profile"}).find(name="a"):
                job_post.append(a)
                flag = 1
    if(flag==0):
        job_post.append("None")

In [8]:
def extract_company_from_result(job_div, job_post): 
    flag = 0
    if(job_div.find(name="div", attrs={"class":"individual_internship_header"})):
        if(job_div.find(name="div", attrs={"class":"individual_internship_header"}).find(name="div", attrs={"class":"company"})):
            if(job_div.find(name="div", attrs={"class":"individual_internship_header"}).find(name="div", attrs={"class":"company"}).find(name="div", attrs={"class":"company_name"})):
                for a in job_div.find(name="div", attrs={"class":"individual_internship_header"}).find(name="div", attrs={"class":"company"}).find(name="div", attrs={"class":"company_name"}).find(name="a"):
                    job_post.append(str(a).strip())
                    flag = 1
    if(flag==0):
        job_post.append("None")

In [9]:
def extract_location_from_result(job_div, job_post): 
    flag = 0
    if(job_div.find(name="div", attrs={"class":"individual_internship_details"})):
        if(job_div.find(name="div", attrs={"class":"individual_internship_details"}).find(name="div", attrs={"id":"location_names"})):
            for a in job_div.find(name="div", attrs={"class":"individual_internship_details"}).find(name="div", attrs={"id":"location_names"}).find(name="a", attrs={"class":"location_link"}):
                job_post.append(str(a).strip())
                flag = 1
    if(flag==0):
        job_post.append("None")

In [10]:
def extract_salary_from_result(job_div, job_post): 
    flag = 0
    if(job_div.find(name="div", attrs={"class":"individual_internship_details"})):
        if(job_div.find(name="div", attrs={"class":"individual_internship_details"}).find(name="div", attrs={"class":"internship_other_details_container"})):
            for stipend in job_div.find(name="div", attrs={"class":"individual_internship_details"}).find(name="div", attrs={"class":"internship_other_details_container"}).find(name="span", attrs={"class":"stipend"}):        
                if isinstance(stipend, bs4.element.NavigableString):
                    job_post.append(str(stipend).strip())
                    flag = 1
    if(flag==0):
        job_post.append("None")

In [11]:
def extract_duration_from_result(job_div, job_post): 
    flag = 0
    if(job_div.find(name="div", attrs={"class":"individual_internship_details"})):
        if(job_div.find(name="div", attrs={"class":"individual_internship_details"}).find(name="div", attrs={"class":"internship_other_details_container"})):
            for stipend in job_div.find(name="div", attrs={"class":"individual_internship_details"}).find(name="div", attrs={"class":"internship_other_details_container"}).find(name="span", attrs={"class":"stipend"}):        
                if isinstance(stipend, bs4.element.NavigableString):
                    flag = 1
                    job_post.append(str(stipend).strip())
    if(flag==0):
        job_post.append("None")

In [12]:
def Get_URL_Of_page(base_url, skill):
    return base_url + "/internships/" + skill + "-internship"    

In [13]:
def Get_total_pages(url):
    # get total number of pages
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser", from_encoding="utf-8")
    total_pages = soup.find(name="span", attrs={"id":"total_pages"})
    return int(total_pages.text.strip())

In [14]:
def Get_Internship_Description_Page_Url(job_div, base_url):
    if(job_div.find(name="div", attrs={"class":"button_container_card"})):
        for a in job_div.find(name="div", attrs={"class":"button_container_card"}).find_all(name="a", attrs={"class":"view_detail_button_outline"}, href=True):
            return 'https://internshala.com/' + a['href']
    return 'None'

In [23]:
def extract_description_from_result(job_div, job_post):
    url = Get_Internship_Description_Page_Url(job_div, base_url)
    flag = 0
    if(url!="None"):
        page = requests.get(url)
        soup_desc = BeautifulSoup(page.text, "html.parser", from_encoding="utf-8")
        text = ''
        for div in soup_desc.find_all(name="div", attrs={"class":"text-container"}):
            for e in div.recursiveChildGenerator():
                if isinstance(e, bs4.element.NavigableString):
                    text += e.strip()
                elif e.name == 'br':
                    text += ' '
                flag = 1
        job_post.append(text)
        text1 = ''
        for div in soup_desc.find_all(name="div", attrs={"class":"about_company_text_container"}):
            text1 = div.text
            flag = 1
        if(len(text1.strip())>=15):
            job_post.append(1)
        else:
            job_post.append(0)
        cur = 0
        f1 = 0
        n1 = 0
        n2 = 0
        for div in soup_desc.find_all(name="div", attrs={"class":"activity"}):
            cur = cur+1
            if(cur==1):
                continue
            val = div.text
            if(cur==2):
                n1 = int(re.search(r'\d+', val).group())
            if(cur==3):
                n2 = int(re.search(r'\d+', val).group())
            f1 = f1 + 1
        print(f1)
        if(f1==2):
            job_post.append(n1/n2)
        else:
            job_post.append(0)
        img = soup_desc.find(name="div", attrs={"class":"internship_logo"}).findAll("img", recursive=False)
        if(img==None):
            job_post.append(0)
        else:
            job_post.append(1)
        if(len(text.strip())>=15):
            job_post.append(1)
        else:
            job_post.append(0)
    if(flag==0):
        job_post.append("None")
        job_post.append(0)
        job_post.append(0)
        job_post.append(0)
        job_post.append(0)

In [24]:
def extract_company_description_from_result(div, job_post):
    url = Get_Internship_Description_Page_Url(job_div, base_url)
    flag = 0
    if(url!="None"):
        page = requests.get(url)
        soup_desc = BeautifulSoup(page.text, "html.parser", from_encoding="utf-8")
        text = ''
        for div in soup_desc.find_all(name="div", attrs={"class":"about_company_text_container"}):
            text = div
            flag = 1
        if(len(text.strip())>=10):
            job_post.append(1)
        else:
            job_post.append(0)
    if(flag==0):
        job_post.append(0)

In [25]:
def extract_hired_to_posted_ratio(div, job_post):
    url = Get_Internship_Description_Page_Url(job_div, base_url)
    flag = 0
    if(url!="None"):
        page = requests.get(url)
        soup_desc = BeautifulSoup(page.text, "html.parser", from_encoding="utf-8")
        text = ''
        for div in soup_desc.find_all(name="div", attrs={"class":"about_company_text_container"}):
            text = div.text
            flag = 1
        if(len(text.strip())<10):
            job_post.append(1)
        else:
            job_post.append(0)
    if(flag==0):
        job_post.append(0)

In [33]:
columns = ["job_title", "company_name", "location", "salary", "description", "company_description", "hired_to_posted_ratio", "has_company_logo", "job_description_length"]
#sample_df = pd.DataFrame(columns = columns)
base_url = "https://internshala.com"
skill = "computer-science"
sample_df = []

cur = 0
df3 = pd.DataFrame(columns = columns)
print(df3)
url = Get_URL_Of_page(base_url, skill)
total_pages = Get_total_pages(url)
arr = []
for page_number in range(1,total_pages):
    page = requests.get(url + "/page-" + str(page_number + 1))
    #time.sleep(1)  #ensuring at least 1 second between page grabs
    soup = BeautifulSoup(page.text, "html.parser")

    for div in soup.find_all(name="div", attrs={"class":"individual_internship"}): 
        #specifying row num for index of job posting in dataframe
        num = (cur + 1)
        #creating an empty list to hold the data for each posting
        job_post = []
        #grabbing job title
        extract_job_title_from_result(div, job_post)
        #grabbing company name
        extract_company_from_result(div, job_post)
        #grabbing location name
        extract_location_from_result(div, job_post)
        #grabbing salary
        extract_salary_from_result(div, job_post)
        #grabbing internship description
        extract_description_from_result(div, job_post)
        #appending list of job post info to dataframe at index nu
        df3.loc[cur+1] = job_post
        sample_df.append(job_post)
        print(df3.loc[cur+1])
        cur = cur+1


Empty DataFrame
Columns: [job_title, company_name, location, salary, description, company_description, hired_to_posted_ratio, has_company_logo, job_description_length]
Index: []


/home/akshay/.local/lib/python3.8/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


2
job_title                                               iOS App Development
company_name                                                      STEMpedia
location                                                          Ahmedabad
salary                                                ₹ 9,000-15,000 /month
description               Selected intern's day-to-day responsibilities ...
company_description                                                       1
hired_to_posted_ratio                                              2.733871
has_company_logo                                                          1
job_description_length                                                    1
Name: 1, dtype: object


/home/akshay/.local/lib/python3.8/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


2
job_title                                          Internet Of Things (IoT)
company_name                            Purewatt Renewables Private Limited
location                                                            Gunupur
salary                                                      ₹ 10,000 /month
description               Selected intern's day-to-day responsibilities ...
company_description                                                       1
hired_to_posted_ratio                                                   3.0
has_company_logo                                                          1
job_description_length                                                    1
Name: 2, dtype: object


...

/home/akshay/.local/lib/python3.8/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


2
job_title                                                 Data Science - AI
company_name                                                        Fittlyf
location                                                     Work From Home
salary                                                       ₹ 3,000 /month
description               We are seeking a passionate and driven data sc...
company_description                                                       1
hired_to_posted_ratio                                              3.142857
has_company_logo                                                          1
job_description_length                                                    1
Name: 511, dtype: object


/home/akshay/.local/lib/python3.8/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


2
job_title                                          Flask/Python Development
company_name                             Druidot Consulting Private Limited
location                                                     Work From Home
salary                                                       ₹ 1,000 /month
description               Selected intern's day-to-day responsibilities ...
company_description                                                       1
hired_to_posted_ratio                                                   3.4
has_company_logo                                                          1
job_description_length                                                    1
Name: 512, dtype: object
0
job_title                                              Software Development
company_name                                        Testing - Please Ignore
location                                                             Mumbai
salary                                                     

/home/akshay/.local/lib/python3.8/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [34]:
#df_internships = Scrap_Internshala(base_url, skill)
#df_internships.to_csv('internshala_internship.csv')

,job_title,company_name,location,salary,description
1,Bubble (No Code) Development,Driveo Electric Mobility Private Limited,Delhi,"₹ 10,000 /month",https://internshala.com//internship/detail/bub...


In [152]:
df4 = pd.DataFrame(df_internships)

In [28]:
df3.head()

,job_title,company_name,location,salary,description,company_description,hired_to_posted_ratio,has_company_logo,job_description_length
1,Android App Development,Qatapultt Learning Solutions Private Limited,Work From Home,"₹ 10,000 /month",Selected intern's day-to-day responsibilities ...,1,4.000000,1,1
2,Mobile App Development,Xsoln,Jaipur,"₹ 6,500 /month",Selected intern's day-to-day responsibilities ...,1,0.000000,1,1
3,Coding And Robotics,Makerminds Kochi,Kochi,"₹ 10,000-15,000 /month",Selected intern's day-to-day responsibilities ...,1,0.000000,1,1
4,Quality Analyst,Cognosmed Laboratories Private Limited,Patna,"₹ 5,000 /month",Selected intern's day-to-day responsibilities ...,1,9.000000,1,1
5,WordPress Development,Perfect Designing Solution,Ahmedabad,"₹ 5,000-8,000 /month",We are seeking a talented and enthusiastic Wor...,1,2.666667,1,1


In [35]:
#df3.to_csv(r'./internships_is2.csv')
df3.to_csv('internships_internshala.csv', mode='a', index=False, header=False)

In [34]:
df3.shape

(513, 9)